In [ ]:
##
#Load Packages
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
import sys
from scipy.optimize import curve_fit

In [ ]:
##
#Define Path to Code Database
DirPath = '/Your/Path/To/Code/'

##
#Define Output Path
OutputPath = '/Your/Path/To/Output/'

In [ ]:
##
#Load Functions
sys.path.append(''.join([DirPath,'bin']))
from EPGMotion import *
from EPGForwardMotionModel import *
from Jacobian_MotionCorrection import *
from ParameterOptionsSimulation import *

In [ ]:
##
#Read Parameters
opt = ParameterOptionsSimulation()

In [ ]:
##
#Conventional EPG simulates a maximum k-value pathway equal to the number of TRs. This piece of code identifies where increasing the k-value leads to no meaningful change in the signal, with subsequent thresholding to accelerate modelling
opt["kOrder"] = kOrder(opt.copy())

In [ ]:
##
#Load Monte Carlo Simulation Data (Convert to Complex)
Signal = np.loadtxt(''.join([DirPath,'/MonteCarlo/CardiacMagnitudeMC.csv']),delimiter=',')*np.exp(1j*np.loadtxt(''.join([DirPath,'/MonteCarlo/CardiacPhaseMC.csv']),delimiter=','))

##
#Load Motion Profile - Identical to those used for the MC simulations (every 100th timepoint)
V = np.loadtxt(''.join([DirPath,'/MonteCarlo/MotionProfileCardiac.csv']),delimiter=',')

##
#Multiply Signal by -1 to set the phase offset equal to 0.
Signal *= -1

In [ ]:
##
#Add Noise

##
#Estimate Noise SD
SNR = 2
NoiseSD = np.mean(abs(Signal[int(opt["nDummy"][0]):]))/SNR

##
#Number of Repeats
nRepeats = 10

##
#Initialise SignalNoise Repeats
SignalNoise = np.zeros((Signal.shape[0],nRepeats),dtype = 'c8')

##
#Add Noise
for k in range(SignalNoise.shape[1]):
    SignalNoise[:,k] = (np.real(Signal) + np.random.normal(0, NoiseSD, Signal.shape[0])) + 1j*(np.imag(Signal) + np.random.normal(0, NoiseSD, Signal.shape[0]))

In [ ]:
##
#Define lower and upper parameter bounds (fix S0 equal to 1, define MotionParameters between -1.5 and 1.5 mm/s))
low = [1E-6, 1, -20*np.pi, *np.ones((int(opt["nTR"][0])-int(opt["SteadyStateTR"][0])))*-1.5]
high = [10E-3, 1+ + np.finfo('f8').eps, 20*np.pi, *np.ones((int(opt["nTR"][0])-int(opt["SteadyStateTR"][0])))*1.5]

In [ ]:
##
#Fitting without Motion Estimation

##
#Initialise fitting parameters (D, S0, Phi)
par_init = [5E-4, 1, np.pi/2]

##
#Initialise Array
poptnoMotion = np.zeros((3,SignalNoise.shape[1]))

##
#Perform Fitting
for k in range(SignalNoise.shape[1]):
    #Input Data (1D array: Real & Imaginary Components - Fitting to After Dummy Region)
    Data = np.concatenate((np.real(SignalNoise[int(opt["nDummy"][0]):,k]).reshape(SignalNoise[int(opt["nDummy"][0]):,k].shape[0]),np.imag(SignalNoise[int(opt["nDummy"][0]):,k]).reshape(SignalNoise[int(opt["nDummy"][0]):,k].shape[0])))
    #Perform Fitting
    poptnoMotion[:,k], pcov, infodict, mesg, ier  = curve_fit(lambda x, *theta: EPGForwardModelFitting(x, theta, opt.copy()), 1, Data, p0=par_init, method='trf',absolute_sigma=False,bounds=(low[0:3],high[0:3]),verbose=1,jac=lambda x, *theta: EPGForwardModelFittingJacobian(x,theta,opt.copy()), x_scale='jac',full_output=True,tr_solver='exact')


In [ ]:
##
#Fitting with Motion Estimation

##
#Initialise fitting parameters (D, S0, Phi, MotionVector)
par_init = [0.5E-4, 1, np.pi/2, *np.zeros(int(opt["nTR"][0])-int(opt["SteadyStateTR"][0]))]

##
#Initialise Array
poptMotion = np.zeros((len(low),SignalNoise.shape[1]))

##
#Perform Fitting
for k in range(SignalNoise.shape[1]):
    #Input Data (1D array: Real & Imaginary Components - Fitting to After Dummy Region)
    Data = np.concatenate((np.real(SignalNoise[int(opt["nDummy"][0]):,k]).reshape(SignalNoise[int(opt["nDummy"][0]):,k].shape[0]),np.imag(SignalNoise[int(opt["nDummy"][0]):,k]).reshape(SignalNoise[int(opt["nDummy"][0]):,k].shape[0])))
    #Perform Fitting
    poptMotion[:,k], pcov, infodict, mesg, ier  = curve_fit(lambda x, *theta: EPGForwardMotionModelFitting(x, theta, opt.copy()), 1, Data, p0=par_init, method='trf',absolute_sigma=False,bounds=(low,high),verbose=1,jac=lambda x, *theta: EPGForwardMotionModelFittingJacobian(x,theta,opt.copy()), x_scale='jac',full_output=True,tr_solver='exact',ftol=1e-3, xtol=1e-3, gtol=1e-3)

In [ ]:
##
#Reconstruct Signal & Motion Profile

##
#Create Reconstructed Motion Array
VRecon = np.zeros_like(SignalNoise, dtype = 'f8')
VRecon[-(int(opt["nTR"][0])-int(opt["SteadyStateTR"][0])):,:] = poptMotion[3:]

##
#Initialise Signal Array
SignalRecon = np.zeros_like(SignalNoise, dtype = 'c8')

##
#Reconstruct Signal
for k in range(SignalNoise.shape[1]):
    ##
    #Declare outputs from fitting to the options file 
    optRecon = opt.copy()
    optRecon['D'] = np.asarray([poptMotion[0,k]], dtype='f8')   
    optRecon['phi'] = np.asarray([poptMotion[2,k]], dtype='f8') 

    ##
    #Reconstruct signal
    SignalRecon[:,k] = EPGMotion(optRecon.copy(), VRecon[:,k])

In [ ]:
##
#Get Mean & Standard Deviation (Original Signal)
SignalNoiseMean = np.mean(SignalNoise,axis=-1)
SignalNoiseSD = np.std(SignalNoise,axis=-1)

##
#Get Mean & Standard Deviation (Reconstructed Signal)
SignalReconMean = np.mean(SignalRecon,axis=-1)
SignalReconSD = np.std(SignalRecon,axis=-1)

##
#Get Mean & Standard Deviation (Motion Profile)
VReconMean = np.mean(VRecon,axis=-1)
VReconSD = np.std(VRecon,axis=-1)

In [ ]:
##
#Plot 
fig, axs = plt.subplots(3, 1)
fig.set_size_inches(8,12)

##
#Define x-axis
Time = range(Signal.shape[0])*opt["TR"]/1E3

##
#Plot Magnitude data
axs[0].plot(Time, np.abs(SignalNoiseMean),'#1f77b4',linewidth=2,label = 'Data (Monte Carlo)')
axs[0].plot(Time,np.abs(SignalReconMean),'#ff7f0e',linewidth=2,linestyle='--',label = 'Fit (EPG + Motion)' )
axs[0].fill_between(Time,np.abs(SignalReconMean)+np.abs(SignalReconSD),np.abs(SignalReconMean)-np.abs(SignalReconSD), color='#ff7f0e',alpha=0.2)
axs[0].axvspan(Time[int(opt["SteadyStateTR"][0])],Time[int(opt["nDummy"][0])], alpha=0.1,color='#d62728',label = 'Dummy Measurements')
axs[0].axvspan(Time[int(opt["nDummy"][0])],Time[-1],alpha=0.1,color='#2ca02c',label = 'Measured Data')

##
#Plot Phase data (multiply by -i to characterise the motion-free phase as 0)
axs[1].plot(Time, np.angle(SignalNoiseMean*-1),'#1f77b4',linewidth=2)
axs[1].plot(Time,np.angle(SignalReconMean*-1),'#ff7f0e',linewidth=2,linestyle='--')
axs[1].axvspan(Time[int(opt["SteadyStateTR"][0])],Time[int(opt["nDummy"][0])], alpha=0.1,color='#d62728')
axs[1].axvspan(Time[int(opt["nDummy"][0])],Time[-1],alpha=0.1,color='#2ca02c')
axs[1].fill_between(Time,np.angle((SignalReconMean+SignalReconSD)*-1),np.angle((SignalReconMean-SignalReconSD)*-1), color='#ff7f0e',alpha=0.2)

##
#Plot Time-Series Data
axs[2].plot(Time,V,'#1f77b4',linewidth=2)
axs[2].plot(Time,VReconMean,'#ff7f0e',linewidth=2,linestyle='--')
axs[2].fill_between(Time,VReconMean+VReconSD,VReconMean-VReconSD, color='#ff7f0e',alpha=0.2)
axs[2].axvspan(Time[int(opt["SteadyStateTR"][0])],Time[int(opt["nDummy"][0])], alpha=0.1,color='#d62728')
axs[2].axvspan(Time[int(opt["nDummy"][0])],Time[-1],alpha=0.1,color='#2ca02c')

##
#Add labels etc
fig.subplots_adjust(hspace=0.4)
axs[0].text(-0.1, 1.05, '(a)', transform=axs[0].transAxes, size=20)
axs[2].text(-0.1, 1.05, '(b)', transform=axs[2].transAxes, size=20)
axs[0].set_xlim([0,Time[-1]])
axs[1].set_xlim([0,Time[-1]])
axs[2].set_xlim([0,Time[-1]])
axs[0].set_ylim([0,0.05])
axs[1].set_ylim([-np.pi,np.pi])
axs[2].set_ylim([-0.3,0.7])
axs[0].set_ylabel('Amplitude (a.u.)',fontsize=12)
axs[1].set_ylabel('Angle (rad.)',fontsize=12)
axs[2].set_ylabel('Velocity (mm/s)',fontsize=12)
axs[0].set_xlabel('Time (s)',fontsize=12)
axs[1].set_xlabel('Time (s)',fontsize=12)
axs[2].set_xlabel('Time (s)',fontsize=12)
axs[0].set_title('Magnitude',fontsize=16)
axs[1].set_title('Phase',fontsize=16)
axs[2].set_title(r'$V(t)$',fontsize=16)
axs[1].set_yticks([-np.pi,0, np.pi],[r'-$\pi$',0, r'$\pi$'])
axs[0].legend(fontsize=10)

In [ ]:
##
#Save Figure
fig.savefig(''.join([OutputPath,'FigureS5.png']),dpi=300,format='png',bbox_inches='tight')